In [ ]:
import requests
import pprint

# SharePoint site URL
# site_url = "https://your-sharepoint-site-url/
site_url= "https://mngenvmcap277115.sharepoint.com/sites/NasaFiles/"

# The SharePoint folder whose files you want to list
folder_url = "/Shared Documents"
client_id = ""
tenant_id = ""
client_secret = ""

site_id = ''  # replace with your actual site id  https://graph.microsoft.com/v1.0/sites?search=contoso
drive_id = ''  # replace with your actual drive id  https://graph.microsoft.com/v1.0//sites/{siteId}/drives


In [ ]:
# Get an access token - Graph API
def get_access_token(tenant_id, client_id, client_secret):
    url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded'
    }

    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'https://graph.microsoft.com/.default'
    }

    response = requests.post(url, headers=headers, data=data)
    response.raise_for_status()  # raises an exception if the request fails

    return response.json()['access_token']


access_token = get_access_token(tenant_id, client_id, client_secret)
print(access_token)



In [ ]:
#Recursively list all files from a SharePoint folder
def list_files(site_id, drive_id, item_id='', files=[]):

    access_token = get_access_token(tenant_id, client_id, client_secret)       
    print("access_token: " + access_token)
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
       }

    if item_id:
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/items/{item_id}/children"
    else:
        url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/root/children"

  
    response = requests.get(url, headers=headers)
    data = response.json()  
    print(data) 

    values =  data['value'] 

    for item in values:        
        if 'folder' in item:
            item_id = item['id']                       
            files = list_files(site_id, drive_id, item_id, files)
        else:           
            files.append(item)

    return files

# usage
files = list_files(site_id, drive_id)
for file in files:   
    print(file['name'])


In [ ]:
def download_file(site_id, drive_id, item_id,file_name):

    access_token = get_access_token(tenant_id, client_id, client_secret)       

    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {access_token}'
    }

    
    url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives/{drive_id}/items/{item_id}/content"
   

    response = requests.get(url, headers=headers)
   
    # Save file
    with open(file_name, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)      
 # usage

download_url = files[0]['@microsoft.graph.downloadUrl']
print(download_url)
pprint(files[0])
item_id = files[0]['id']
file_name = files[0]['name']
download_file(site_id, drive_id, item_id, file_name)
   